<a href="https://colab.research.google.com/github/Snafkin547/Quantum-ML/blob/main/Quantum_ML_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit
!pip install numpy scipy matplotlib ipython pandas sympy nose seaborn
!pip install scikit-learn
!pip install pylatexenc ipywidgets qutip
!pip install kaggle

from google.colab import drive
import os

#Google Drive mounting to Google Colab
drive.mount('/content/gdrive')
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/QML/Kaggle"
#Change the working directory
%cd /content/gdrive/My Drive/QML/Kaggle/

#Check if the directory was properly changed
%pwd

import pandas as pd

train=pd.read_csv('./train.csv')

In [2]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Convenient Functions to prepare

In [3]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, result
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from math import asin, sqrt, sin, cos, pi

def ccry(qc, theta, control1, control2, controlled):
  qc.cry(theta/2, control2, controlled)
  qc.cx(control1, control2)
  qc.cry(-theta/2, control2, controlled)
  qc.cx(control1, control2)
  qc.cry(theta/2, control1, controlled)

def marg_prob(obj):
  prob=len(obj)/len(train)
#  print(prob)
  return prob

def prob_to_angle(prob):
  return 2*asin(sqrt(prob))

def apply_prob(qc, prob, pos):
  qc.ry(prob_to_angle(prob), pos)

def apply_ischild_sex(qc):
  apply_prob(qc, prob_child, ISCHILD_POS)
  apply_prob(qc, prob_female, SEX_POS)

def apply_norm(qc, norm_params):
  # Set conditional prob for male/adult  
  qc.x(ISCHILD_POS)
  qc.x(SEX_POS)
  ccry(qc, prob_to_angle(norm_params['p_norm_am']), ISCHILD_POS, SEX_POS, QPOS_NORM)
  qc.x(ISCHILD_POS)
  qc.x(SEX_POS)
  # Set conditional prob for female/adult  
  qc.x(ISCHILD_POS)
  ccry(qc, prob_to_angle(norm_params['p_norm_af']), ISCHILD_POS, SEX_POS, QPOS_NORM)
  qc.x(ISCHILD_POS)
  # Set conditional prob for male/child  
  qc.x(SEX_POS)
  ccry(qc, prob_to_angle(norm_params['p_norm_cm']), ISCHILD_POS, SEX_POS, QPOS_NORM)
  qc.x(SEX_POS)
  # Set conditional prob for female/child  
  ccry(qc, prob_to_angle(norm_params['p_norm_cf']), ISCHILD_POS, SEX_POS, QPOS_NORM)

def apply_class(qc):
  qc.ry(prob_to_angle(prob_first), FIRST_POS)
  qc.x(FIRST_POS)
  qc.cry(prob_to_angle(prob_second/(1-prob_first)), FIRST_POS, SECOND_POS)
  qc.x(SECOND_POS)
  ccry(qc, prob_to_angle(prob_third/(1-prob_first-prob_second)), FIRST_POS, SECOND_POS, THIRD_POS)
  qc.x(SECOND_POS)
  qc.x(FIRST_POS)

def apply_survival(qc, surv_params):

  qc.x(QPOS_NORM)
  ccry(qc, prob_to_angle(surv_params['p_surv_u1']), QPOS_NORM, FIRST_POS, SURV_POS)
  ccry(qc, prob_to_angle(surv_params['p_surv_u2']), QPOS_NORM, SECOND_POS, SURV_POS)
  ccry(qc, prob_to_angle(surv_params['p_surv_u3']), QPOS_NORM, THIRD_POS, SURV_POS)
  qc.x(QPOS_NORM)
  ccry(qc, prob_to_angle(surv_params['p_surv_f1']), QPOS_NORM, FIRST_POS, SURV_POS)
  ccry(qc, prob_to_angle(surv_params['p_surv_f2']), QPOS_NORM, SECOND_POS, SURV_POS)
  ccry(qc, prob_to_angle(surv_params['p_surv_f3']), QPOS_NORM, THIRD_POS, SURV_POS)

def prepare_data(passengers, params):
  #Create IsChild Col
  passengers['IsChild']=passengers['Age'].map(lambda age:0 if age >max_child_age else 1)

  #Prob of favored by Norm given Age, Sex, and survival
  passengers['Norm']= list(map(lambda item: params['p_norm_{}{}{}'.format(
      'a' if item[0]==0 else 'c',
      item[1][0],
      'd' if item[2]==0 else 's'
      )],
      list(zip(passengers['IsChild'], passengers['Sex'], passengers['Survived']))
      ))
  return passengers

def calculate_norm_params(passengers):
  pop_children=passengers[passengers['IsChild']==1]
  pop_adults=passengers[passengers['IsChild']==0]
  pop_am=pop_adults[pop_adults['Sex']=='male']
  pop_af=pop_adults[pop_adults['Sex']=='female']
  pop_cm=pop_children[pop_children['Sex']=='male']
  pop_cf=pop_children[pop_children['Sex']=='female']

  norm_params={
      'p_norm_am': pop_am['Norm'].sum()/len(pop_am),
      'p_norm_af': pop_af['Norm'].sum()/len(pop_af),
      'p_norm_cm': pop_cm['Norm'].sum()/len(pop_cm),
      'p_norm_cf': pop_cf['Norm'].sum()/len(pop_cf)
  }
  return norm_params

def calculate_surv_params(passengers):
  survivors=passengers[passengers['Survived']==1]
  
  def weight_passenger(norm, pclass):
    return lambda passenger: (passenger[0] if norm==True else 1-passenger[0])*(1 if passenger[1]==pclass else 0)
  
  def calc_prob(norm, pclass):
    return sum(list(map(
        weight_passenger(norm, pclass), 
        list(zip(survivors['Norm'], survivors['Pclass']))
        )))/sum(list(map(weight_passenger(norm, pclass), list(zip(passengers['Norm'], passengers['Pclass']))
        )))

  surv_params={
      'p_surv_f1': calc_prob(True, 1),
      'p_surv_f2': calc_prob(True, 2),
      'p_surv_f3': calc_prob(True, 3),
      'p_surv_u1': calc_prob(False, 1),
      'p_surv_u2': calc_prob(False, 2),
      'p_surv_u3': calc_prob(False, 3)
  }
  return surv_params

## Summary functions
def filter_states(states, position, value):
  return list(filter(lambda item: item[0][QUBITS-1-position]==str(value), states))
def sum_states(states):
  return sum(map(lambda item: item[1], states))

### Define model and Train Norm-Parameter

In [4]:
## Q-ML Model
def qbn_model(QUBITS, with_qc=None, norm_params=None, surv_params=None, hist=False, measure=False, shots=1):
  if measure:
      qr=QuantumRegister(QUBITS)
      cr=ClassicalRegister(1)
      qc=QuantumCircuit(qr,cr)
  else:
    qc=QuantumCircuit(QUBITS)
    
  if with_qc!=None:
    with_qc(qc, qr=qr, cr=cr)
  elif norm_params!=None and surv_params!=None:
    apply_ischild_sex(qc)
    apply_class(qc)
    apply_norm(qc, norm_params)
    apply_survival(qc, surv_params)

  if measure:
    qc.measure(qr[SURV_POS], cr[0]) 

  results=execute(qc, Aer.get_backend('statevector_simulator') if measure is False else Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
  return results

# Train norm params
def train_qbn(passengers, params, iterations):
    ## Refine norm_parameters
    def to_params(results):
        states=results.items()
        def calc_norm(ischild_val, sex_val, surv_val):
            pop=filter_states(filter_states(filter_states(states, ISCHILD_POS, ischild_val), SEX_POS, sex_val), SURV_POS, surv_val)
            p_norm=sum(map(lambda item: item[1], filter_states(pop, QPOS_NORM, '1')))
            p_total=sum(map(lambda item:item[1], pop))
            return p_norm/p_total
        return {
            'p_norm_cms': calc_norm('1', '0', '1'),
            'p_norm_cmd': calc_norm('1', '0', '0'),
            'p_norm_cfs': calc_norm('1', '1', '1'),
            'p_norm_cfd': calc_norm('1', '1', '0'),
            'p_norm_ams': calc_norm('0', '0', '1'),
            'p_norm_amd': calc_norm('0', '0', '0'),
            'p_norm_afs': calc_norm('0', '1', '1'),
            'p_norm_afd': calc_norm('0', '1', '0'),
        }
    # Run recursively to refin norm_params
    if iterations>0:
        new_params=train_qbn(passengers, params, iterations-1)
        passengers=prepare_data(passengers, new_params)
        norm_params=calculate_norm_params(passengers)
        surv_params=calculate_surv_params(passengers)
        results=qbn_model(QUBITS, norm_params=norm_params, surv_params=surv_params, hist=False, measure=False, shots=1000)
        return to_params(results)
    return params

### Prediction

In [7]:
## pre-process to Return 
def pre_process(passenger):
    return (passenger['IsChild'] == 1, passenger['Sex'] == 'female', passenger['Pclass'])

## get a qbn using norm_params trained in the previous sec
def get_trained_qbn(passengers, params):
    prepared_passengers = prepare_data(passengers, params)
    norm_params = calculate_norm_params(prepared_passengers)
    surv_params = calculate_surv_params(prepared_passengers)
    
    ## This is to formulate a QBN to predict and with each passenger data fed into at the Evaluation phase
    def trained_qbn_titanic(passenger):
        (is_child, is_female, pclass) = passenger

        def apply_known(qc, is_child, is_female, pclass):
          if is_child:
            qc.x(ISCHILD_POS)
          
          if is_female:
            qc.x(SEX_POS)
          
          qc.x(FIRST_POS if pclass==1 else (SECOND_POS if pclass==2 else THIRD_POS))

        def circuit(qc, qr, cr):
            apply_known(qc, is_child, is_female, pclass)
            apply_norm(qc, norm_params)
            apply_survival(qc, surv_params)
            qc.measure(qr[SURV_POS], cr[0])
        # Prediction for each passenger
        return qbn_model(QUBITS, with_qc=circuit, norm_params=None, surv_params=None, hist=False, measure=True, shots=100)
    return trained_qbn_titanic

### Evaluation

In [5]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score
def run(f_classify, x):
    return list(map(f_classify, x))

def specificity(matrix):
    return matrix[0][0]/(matrix[0][0]+matrix[0][1]) if (matrix[0][0]+matrix[0][1] > 0) else 0

def npv(matrix):
    return matrix[0][0]/(matrix[0][0]+matrix[1][0]) if (matrix[0][0]+matrix[1][0] > 0) else 0

def classifier_report(name, run, classify, input, labels):
    cr_predictions = run(classify, input)
    cr_cm = confusion_matrix(labels, cr_predictions)

    cr_precision = precision_score(labels, cr_predictions)
    cr_recall = recall_score(labels, cr_predictions)
    cr_specificity = specificity(cr_cm)
    cr_npv = npv(cr_cm)
    cr_level = 0.25*(cr_precision + cr_recall + cr_specificity + cr_npv)

    print('The precision score of the {} classifier is {:.2f}'
        .format(name, cr_precision))
    print('The recall score of the {} classifier is {:.2f}'
        .format(name, cr_recall))
    print('The specificity score of the {} classifier is {:.2f}'
        .format(name, cr_specificity))
    print('The npv score of the {} classifier is {:.2f}'
        .format(name, cr_npv))
    print('The information level is: {:.2f}'
        .format(cr_level))
    
def run(f_classify, data):
    return [f_classify(data.iloc[i]) for i in range(0,len(data))]

def post_process(counts):
  p_surv=counts['1'] if '1' in counts.keys() else 0
  p_died=counts['0'] if '0' in counts.keys() else 0
  return 1 if p_surv>p_died else 0

### Implementation

In [6]:
first=train[train["Pclass"]==1]
second=train[train["Pclass"]==2]
third=train[train["Pclass"]==3]
surv=train[train["Survived"]==1]

# the maximum age of a passenger we consider as a child
max_child_age = 8

# probability of being a child
child = train[train.Age.le(max_child_age)]
p_child = len(child)/len(train)

# probability of being female
female = train[train.Sex.eq("female")]
p_female = len(female)/len(train)

QUBITS=7
ISCHILD_POS=0
SEX_POS=1
QPOS_NORM=2
FIRST_POS=3
SECOND_POS=4
THIRD_POS=5
SURV_POS=6

prob_surv=marg_prob(surv)
prob_child=marg_prob(child)
prob_female=marg_prob(female)

prob_first=marg_prob(first)
prob_second=marg_prob(second)
prob_third=marg_prob(third)
prob_surv_first=round(len(first[first['Survived']==1])/len(first),2)
prob_surv_second=round(len(second[second['Survived']==1])/len(first),2)
prob_surv_third=round(len(third[third['Survived']==1])/len(first),2)

arbitrary_initial_param={
 'p_norm_cms': 0.45,
 'p_norm_cmd': 0.46,
 'p_norm_cfs': 0.47,
 'p_norm_cfd': 0.48,
 'p_norm_ams': 0.49,
 'p_norm_amd': 0.51,
 'p_norm_afs': 0.52,
 'p_norm_afd': 0.53,
}
### Step 1) Prepare a dataset
passengers=prepare_data(train, arbitrary_initial_param)

### Step 2) Learning phase, find opt parameters (by iterating 25 times)
trained_params=train_qbn(train, arbitrary_initial_param, 25)

### Step 3) Establish a QBN to predict
trained_qbn = get_trained_qbn(train, trained_params)

# Evaluate the QBN
classifier_report("QBN",
    run,
    lambda passenger: post_process(trained_qbn(pre_process(passenger))),  # Predicting passengers one by one through Step 3
    passengers,
    train['Survived'])


In [15]:
#Iterating 50 times

first=train[train["Pclass"]==1]
second=train[train["Pclass"]==2]
third=train[train["Pclass"]==3]
surv=train[train["Survived"]==1]

# the maximum age of a passenger we consider as a child
max_child_age = 8

# probability of being a child
child = train[train.Age.le(max_child_age)]
p_child = len(child)/len(train)

# probability of being female
female = train[train.Sex.eq("female")]
p_female = len(female)/len(train)

QUBITS=7
ISCHILD_POS=0
SEX_POS=1
QPOS_NORM=2
FIRST_POS=3
SECOND_POS=4
THIRD_POS=5
SURV_POS=6

prob_surv=marg_prob(surv)
prob_child=marg_prob(child)
prob_female=marg_prob(female)

prob_first=marg_prob(first)
prob_second=marg_prob(second)
prob_third=marg_prob(third)
prob_surv_first=round(len(first[first['Survived']==1])/len(first),2)
prob_surv_second=round(len(second[second['Survived']==1])/len(first),2)
prob_surv_third=round(len(third[third['Survived']==1])/len(first),2)

arbitrary_initial_param={
 'p_norm_cms': 0.45,
 'p_norm_cmd': 0.46,
 'p_norm_cfs': 0.47,
 'p_norm_cfd': 0.48,
 'p_norm_ams': 0.49,
 'p_norm_amd': 0.51,
 'p_norm_afs': 0.52,
 'p_norm_afd': 0.53,
}
### Step 1) Prepare a dataset
passengers=prepare_data(train, arbitrary_initial_param)

### Step 2) Learning phase, find opt parameters (by iterating 50 times)
trained_params=train_qbn(train, arbitrary_initial_param, 50)

### Step 3) Establish a QBN to predict
trained_qbn = get_trained_qbn(train, trained_params)

# Evaluate the QBN
classifier_report("QBN",
    run,
    lambda passenger: post_process(trained_qbn(pre_process(passenger))),  # Predicting passengers one by one through Step 3
    passengers,
    train['Survived'])


The precision score of the QBN classifier is 0.82
The recall score of the QBN classifier is 0.60
The specificity score of the QBN classifier is 0.92
The npv score of the QBN classifier is 0.79
The information level is: 0.78
